In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA


In [ ]:
load_dotenv(override=True)
os.environ.get("OPENAI_API_KEY")

In [ ]:
files = ["D:\Visual Code\RAG\pdf\entrecomp.pdf"]
pages = []

In [ ]:
for file in files:
    loader = PyPDFLoader(file)
    pages.extend(loader.load())

In [ ]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [ ]:
documents = recur_split.split_documents(pages)

In [ ]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('pdf/', '')
    doc.metadata['doc_id'] = i

In [ ]:
directory = '../vector/chroma_retrival_bd'

In [ ]:
embeddings_model = OpenAIEmbeddings()

In [ ]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=directory
)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

In [ ]:
question = "O que é Pensamento ético e sustentável?"

In [ ]:
chat_chain.invoke({"query": question})

# Engenharia de Prompt